In [ ]:
%matplotlib inline
import pandas as pd
import pyspark
from pyspark.sql import SQLContext,SparkSession
sc = pyspark.SparkContext(appName="MiniProjAvni")
spark = SparkSession.builder.appName("MiniProjAvni").getOrCreate()
sqlContext = SQLContext(spark)

In [ ]:
df = sqlContext.read.csv('/user/avnidas_gmail/Churn.csv',header='True',inferSchema='true')

In [ ]:
df.cache()
df.printSchema()

In [ ]:
df.show()

In [ ]:
df.filter(df['Account Length'] > 0).show()

In [ ]:
df.groupBy("State").count().show()

In [ ]:
df.createOrReplaceTempView("TelecomChurn")
sqlDF = spark.sql("SELECT * FROM TelecomChurn")
sqlDF

In [ ]:
pd.DataFrame(df.take(15), columns=df.columns).transpose()

In [ ]:
df.describe()

In [ ]:
df.describe().toPandas().transpose()

In [ ]:
new_df = df.toPandas()

In [ ]:
data = new_df.copy()

In [ ]:
features = ['Account Length', 'VMail Message', 'Day Mins','Eve Mins','Night Mins','Intl Mins','CustServ Calls',\
            'Intl Plan','VMail Plan','Day Calls','Eve Calls','Night Calls','Intl Calls','Churn']
Num_features = list(set(new_df.columns) - set(['Intl Plan', 'VMail Plan', 'Churn','CustServ Calls']))
new_df[Num_features].hist(figsize=(16, 12));

#### Area Code and Voice Mail Messages donot follow Gaussian distribution

In [ ]:
col_to_drop = ['State', 'Day Charge', 'Eve Charge', 'Night Charge','Intl Charge']
new_df = df.toPandas().drop(col_to_drop, axis=1)
new_df.head().transpose()

In [ ]:
new_df.describe()

In [ ]:
features = ['Day Mins', 'Intl Calls']
data[features].hist(figsize=(12, 4));

In [ ]:
data[features].plot(kind='density', subplots=True, 
                  layout=(1, 2), sharex=False, figsize=(12, 4));

#### In the above plots, we see that the variable Total day minutes is normally distributed, while Total intl calls is prominently skewed right (its tail is longer on the right).

In [ ]:
data['Churn'].value_counts()

#### The data is not balanced, our two target classes, loyal and disloyal customers, are not represented equally in the dataset. Only a small part of the clients canceled their subscription to the telecom service.

In [ ]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int' or t[1] == 'double']
sampled_data = df.select(numeric_features).sample(False, 0.10).toPandas()

In [ ]:
axs = pd.scatter_matrix(sampled_data, figsize=(16, 16));

# Rotate axis labels and remove axis ticks
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())


In [ ]:
pd.crosstab(data['State'], data['Churn']).T

In [ ]:
import numpy as np
data.groupby(['State'])['Churn'].agg([np.mean]).sort_values(by='mean', ascending=False).T

### At first glance, it can be inferred that the churn rate in New Jersey and California are above 25% and less than 6% for Hawaii and Alaska from the sample data we have

#### Intial Hypothesis: 

**Null hypothesis** of MANOVA is that all the means of the independent variables are equal, which implies that the independent variables are not differentiators of the group.

**Alternative hypothesis** is that at least one independent variable has a different mean or, in other words, a significant differentiator. 


In [ ]:
spark.stop()